In [ ]:
import cv2
import torch
import torch.nn as nn
import numpy as np
from pathlib import Path
from tqdm import tqdm

# =====================
# CONFIG
# =====================
VIDEO_PATH = Path("../data/raw_videos/WhatsApp Video 2026-01-08 at 15.55.48.mp4")

LOWLIGHT_MODEL_PATH = Path("../outputs/models/low_light/best_model_lowlight.pth")
DEBLUR_MODEL_PATH   = Path("../outputs/models/deblur/best_model_deblur.pth")

OUTPUT_PATH = Path("FINAL_lowlight_then_deblur.mp4")

FRAME_SKIP = 10       # 🔥 SAFE SPEEDUP
DEFAULT_FPS = 50

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("[INFO] Device:", DEVICE)

# =====================
# MODELS (MATCH TRAINING)
# =====================

class ZeroDCE(nn.Module):
    def __init__(self):
        super().__init__()
        self.relu = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 32, 3, padding=1)
        self.conv4 = nn.Conv2d(32, 32, 3, padding=1)
        self.conv5 = nn.Conv2d(32, 32, 3, padding=1)
        self.conv6 = nn.Conv2d(32, 32, 3, padding=1)
        self.conv7 = nn.Conv2d(32, 24, 3, padding=1)

    def forward(self, x):
        x1 = self.relu(self.conv1(x))
        x2 = self.relu(self.conv2(x1))
        x3 = self.relu(self.conv3(x2))
        x4 = self.relu(self.conv4(x3))
        x5 = self.relu(self.conv5(x4))
        x6 = self.relu(self.conv6(x5))
        return torch.tanh(self.conv7(x6))

def apply_curves(img, curves, n=8):
    out = img
    for i in range(n):
        r = curves[:, i*3:(i+1)*3]
        out = out + r * (out**2 - out)
    return torch.clamp(out, 0, 1)

class ResBlock(nn.Module):
    def __init__(self, c):
        super().__init__()
        self.conv1 = nn.Conv2d(c, c, 3, padding=1)
        self.conv2 = nn.Conv2d(c, c, 3, padding=1)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        return x + self.conv2(self.relu(self.conv1(x)))

class DeblurGenerator(nn.Module):
    def __init__(self):
        super().__init__()
        self.head = nn.Conv2d(3, 64, 7, padding=3)
        self.body = nn.Sequential(*[ResBlock(64) for _ in range(9)])
        self.tail = nn.Conv2d(64, 3, 7, padding=3)

    def forward(self, x):
        x = torch.relu(self.head(x))
        x = self.body(x)
        return torch.sigmoid(self.tail(x))

# =====================
# LOAD MODELS (SAFE)
# =====================
lowlight = ZeroDCE().to(DEVICE)
low_ckpt = torch.load(LOWLIGHT_MODEL_PATH, map_location=DEVICE, weights_only=True)
lowlight.load_state_dict(low_ckpt["model_state"])
lowlight.eval()


deblur = DeblurGenerator().to(DEVICE)
deblur_ckpt = torch.load(DEBLUR_MODEL_PATH, map_location=DEVICE, weights_only=True)
deblur.load_state_dict(deblur_ckpt["model_state"])
deblur.eval()


print("[INFO] Models loaded successfully")

# =====================
# VIDEO SETUP
# =====================
cap = cv2.VideoCapture(str(VIDEO_PATH))
assert cap.isOpened(), "❌ Cannot open video"

fps = cap.get(cv2.CAP_PROP_FPS)
fps = fps if fps > 1 else DEFAULT_FPS
out_fps = fps / FRAME_SKIP

ret, frame = cap.read()
assert ret, "❌ No frames"

h, w = frame.shape[:2]

writer = cv2.VideoWriter(
    str(OUTPUT_PATH),
    cv2.VideoWriter_fourcc(*"mp4v"),
    out_fps,
    (w * 2, h)
)

cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

# =====================
# FINAL PIPELINE
# =====================
frame_id = 0
processed = 0

print("[INFO] Starting FINAL pipeline inference...")

with torch.no_grad():
    for _ in tqdm(range(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))):

        ret, frame = cap.read()
        if not ret:
            break

        if frame_id % FRAME_SKIP != 0:
            frame_id += 1
            continue

        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        inp = torch.from_numpy(rgb / 255.0)\
                    .permute(2,0,1)\
                    .unsqueeze(0)\
                    .float()\
                    .to(DEVICE)

        # ---- LOW LIGHT ----
        curves = lowlight(inp)
        enhanced = apply_curves(inp, curves)

        # ---- DEBLUR ----
        restored = deblur(enhanced)

        out = restored[0].permute(1,2,0).cpu().numpy()
        out = np.clip(out * 255, 0, 255).astype(np.uint8)
        out_bgr = cv2.cvtColor(out, cv2.COLOR_RGB2BGR)

        combined = np.hstack([frame, out_bgr])
        writer.write(combined)

        processed += 1
        frame_id += 1

cap.release()
writer.release()

print("[DONE]")
print(f"[INFO] Frames processed: {processed}")
print(f"[INFO] Output saved to: {OUTPUT_PATH}")


In [ ]:
import cv2
import torch
import torch.nn as nn
import numpy as np
from pathlib import Path
from tqdm import tqdm

# =====================
# CONFIG
# =====================
VIDEO_PATH = Path("../data/raw_videos/high_speed_axis_55kmph_night.mp4")
DEBLUR_MODEL_PATH = Path("../outputs/models/deblur/best_model_deblur.pth")
OUTPUT_PATH = Path("deblur_only_output.mp4")

FRAME_SKIP = 2.5        # speed-up (process every 5th frame)
DEFAULT_FPS = 50

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("[INFO] Device:", DEVICE)

# =====================
# MODEL (MATCHES TRAINING)
# =====================
class ResBlock(nn.Module):
    def __init__(self, c):
        super().__init__()
        self.conv1 = nn.Conv2d(c, c, 3, padding=1)
        self.conv2 = nn.Conv2d(c, c, 3, padding=1)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        return x + self.conv2(self.relu(self.conv1(x)))

class DeblurGenerator(nn.Module):
    def __init__(self):
        super().__init__()
        self.head = nn.Conv2d(3, 64, 7, padding=3)
        self.body = nn.Sequential(*[ResBlock(64) for _ in range(9)])
        self.tail = nn.Conv2d(64, 3, 7, padding=3)

    def forward(self, x):
        x = torch.relu(self.head(x))
        x = self.body(x)
        return torch.sigmoid(self.tail(x))

# =====================
# LOAD MODEL (CORRECT)
# =====================
model = DeblurGenerator().to(DEVICE)
ckpt = torch.load(DEBLUR_MODEL_PATH, map_location=DEVICE, weights_only=True)
model.load_state_dict(ckpt["model_state"])
model.eval()

print("[INFO] Deblur model loaded successfully")

# =====================
# VIDEO SETUP
# =====================
cap = cv2.VideoCapture(str(VIDEO_PATH))
assert cap.isOpened(), "❌ Cannot open video"

fps = cap.get(cv2.CAP_PROP_FPS)
fps = fps if fps > 1 else DEFAULT_FPS

ret, frame = cap.read()
assert ret, "❌ No frames in video"

h, w = frame.shape[:2]

writer = cv2.VideoWriter(
    str(OUTPUT_PATH),
    cv2.VideoWriter_fourcc(*"mp4v"),
    fps // FRAME_SKIP,
    (w * 2, h)
)

cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

# =====================
# INFERENCE LOOP
# =====================
frame_id = 0
processed = 0

print("[INFO] Starting DEBLUR-ONLY inference...")

with torch.no_grad():
    for _ in tqdm(range(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))):

        ret, frame = cap.read()
        if not ret:
            break

        if frame_id % FRAME_SKIP != 0:
            frame_id += 1
            continue

        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        inp = torch.from_numpy(rgb / 255.0)\
                    .permute(2,0,1)\
                    .unsqueeze(0)\
                    .float()\
                    .to(DEVICE)

        out = model(inp)[0].permute(1,2,0).cpu().numpy()
        out = (out * 255).astype(np.uint8)
        out_bgr = cv2.cvtColor(out, cv2.COLOR_RGB2BGR)

        combined = np.hstack([frame, out_bgr])
        writer.write(combined)

        processed += 1
        frame_id += 1

cap.release()
writer.release()

print("[DONE]")
print(f"[INFO] Frames processed: {processed}")
print(f"[INFO] Output saved to: {OUTPUT_PATH}")


In [ ]:
import cv2
import numpy as np
import onnxruntime as ort
from pathlib import Path
from collections import deque

# ==========================
# CONFIG
# ==========================
VIDEO_PATH = Path("../data/raw_videos/high_speed_axis_55kmph_night.mp4")
OUTPUT_PATH = Path("final_stable_pipeline.mp4")

DEBLUR_ONNX = "../onnx_models/deblur.onnx"
OCR_ONNX    = "../onnx_models/ocr.onnx"

FRAME_SKIP   = 5
OCR_EVERY_N  = 20
DEFAULT_FPS  = 50

EDGE_VAR_THRESH = 80.0
OCR_CONF_THRESH = 0.65
TEMPORAL_WINDOW = 5

ALPHABET = "0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"
PROVIDERS = ["CUDAExecutionProvider", "CPUExecutionProvider"]

# ==========================
# LOAD MODELS
# ==========================
deb_sess = ort.InferenceSession(DEBLUR_ONNX, providers=PROVIDERS)
ocr_sess = ort.InferenceSession(OCR_ONNX, providers=PROVIDERS)

deb_in = deb_sess.get_inputs()[0].name
ocr_in = ocr_sess.get_inputs()[0].name

# ==========================
# VIDEO SETUP
# ==========================
cap = cv2.VideoCapture(str(VIDEO_PATH))
assert cap.isOpened(), "❌ Cannot open video"

fps = cap.get(cv2.CAP_PROP_FPS)
fps = fps if fps > 1 else DEFAULT_FPS
fps /= FRAME_SKIP

ret, first = cap.read()
assert ret, "❌ Empty video"

H, W = first.shape[:2]

writer = cv2.VideoWriter(
    str(OUTPUT_PATH),
    cv2.VideoWriter_fourcc(*"mp4v"),
    fps,
    (W * 2, H)
)

cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

# ==========================
# HELPERS
# ==========================
def softmax(x, axis=-1):
    x = x - np.max(x, axis=axis, keepdims=True)
    e = np.exp(x)
    return e / np.sum(e, axis=axis, keepdims=True)

def decode_ctc(logits):
    probs = softmax(logits, axis=-1)
    preds = probs.argmax(axis=-1)[0]

    prev = -1
    text, confs = "", []

    for t, p in enumerate(preds):
        if p != prev and p != 0:
            text += ALPHABET[p - 1]
            confs.append(probs[0, t, p])
        prev = p

    if not confs:
        return "", 0.0

    return text, float(np.mean(confs))

def get_text_roi(frame):
    h, w = frame.shape[:2]
    return (
        int(w * 0.25),
        int(h * 0.45),
        int(w * 0.75),
        int(h * 0.65),
    )

def has_text_like_content(gray):
    return cv2.Laplacian(gray, cv2.CV_64F).var() > EDGE_VAR_THRESH

# ==========================
# TEMPORAL OCR BUFFER
# ==========================
buffer = deque(maxlen=TEMPORAL_WINDOW)
final_text = ""

# ==========================
# MAIN LOOP
# ==========================
frame_id = 0
processed = 0

print("[INFO] Starting STABLE pipeline...")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    if frame_id % FRAME_SKIP != 0:
        frame_id += 1
        continue

    original = frame.copy()

    # -------------------------
    # DEBLUR (ALWAYS ON)
    # -------------------------
    inp = frame.astype(np.float32) / 255.0
    inp = inp.transpose(2, 0, 1)[None]

    deblurred = deb_sess.run(None, {deb_in: inp})[0][0]
    deblurred = (deblurred.transpose(1, 2, 0) * 255).astype(np.uint8)

    # -------------------------
    # OCR (RAW, NOT DEBLURRED)
    # -------------------------
    if processed % OCR_EVERY_N == 0:
        x1, y1, x2, y2 = get_text_roi(original)
        roi = original[y1:y2, x1:x2]
        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)

        if has_text_like_content(gray):
            gray = cv2.resize(gray, (128, 32))
            ocr_inp = (gray.astype(np.float32) / 255.0)[None, None]

            logits = ocr_sess.run(None, {ocr_in: ocr_inp})[0]
            text, conf = decode_ctc(logits)

            if conf >= OCR_CONF_THRESH:
                buffer.append(text)
                if len(buffer) == TEMPORAL_WINDOW and len(set(buffer)) == 1:
                    final_text = buffer[0]
                    print(f"[OCR ✓] {final_text}")

    # -------------------------
    # VISUALIZATION
    # -------------------------
    vis = deblurred.copy()
    if final_text:
        cv2.putText(
            vis, final_text,
            (30, 50),
            cv2.FONT_HERSHEY_SIMPLEX,
            1.3, (0, 255, 0), 3
        )

    combined = np.hstack([original, vis])
    writer.write(combined)

    processed += 1
    frame_id += 1

cap.release()
writer.release()

print("✅ DONE")


In [ ]:
import cv2
import numpy as np
import onnxruntime as ort
from pathlib import Path
from collections import deque
from tqdm import tqdm

# ==========================
# CONFIG
# ==========================
VIDEO_PATH = Path("../data/raw_videos/high_speed_axis_55kmph_night.mp4")
OUTPUT_PATH = Path("patch_deblur_output.mp4")

DEBLUR_ONNX = "../onnx_models/deblur.onnx"
OCR_ONNX    = "../onnx_models/ocr.onnx"

FRAME_SKIP   = 5
OCR_EVERY_N  = 20
DEFAULT_FPS  = 50

# Patch parameters
TILE_SIZE = 256
OVERLAP   = 32
STEP      = 224

# OCR parameters
EDGE_VAR_THRESH = 80.0
OCR_CONF_THRESH = 0.65
TEMPORAL_WINDOW = 5

ALPHABET = "0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"
PROVIDERS = ["CUDAExecutionProvider", "CPUExecutionProvider"]

# ==========================
# LOAD MODELS
# ==========================
deb_sess = ort.InferenceSession(DEBLUR_ONNX, providers=PROVIDERS)
ocr_sess = ort.InferenceSession(OCR_ONNX, providers=PROVIDERS)

deb_in = deb_sess.get_inputs()[0].name
ocr_in = ocr_sess.get_inputs()[0].name

# ==========================
# VIDEO SETUP
# ==========================
cap = cv2.VideoCapture(str(VIDEO_PATH))
assert cap.isOpened(), "❌ Cannot open video"

total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
total_to_process = total_frames // FRAME_SKIP

fps = cap.get(cv2.CAP_PROP_FPS)
fps = fps if fps > 1 else DEFAULT_FPS
fps /= FRAME_SKIP

ret, first = cap.read()
assert ret, "❌ Empty video"

H, W = first.shape[:2]

writer = cv2.VideoWriter(
    str(OUTPUT_PATH),
    cv2.VideoWriter_fourcc(*"mp4v"),
    fps,
    (W * 2, H)
)

cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

# ==========================
# HELPERS
# ==========================
def softmax(x, axis=-1):
    x = x - np.max(x, axis=axis, keepdims=True)
    e = np.exp(x)
    return e / np.sum(e, axis=axis, keepdims=True)

def decode_ctc(logits):
    probs = softmax(logits, axis=-1)
    preds = probs.argmax(axis=-1)[0]

    prev = -1
    text, confs = "", []

    for t, p in enumerate(preds):
        if p != prev and p != 0:
            text += ALPHABET[p - 1]
            confs.append(probs[0, t, p])
        prev = p

    if not confs:
        return "", 0.0

    return text, float(np.mean(confs))

def has_text_like_content(gray):
    return cv2.Laplacian(gray, cv2.CV_64F).var() > EDGE_VAR_THRESH

def get_text_roi(frame):
    h, w = frame.shape[:2]
    return (
        int(w * 0.25),
        int(h * 0.45),
        int(w * 0.75),
        int(h * 0.65),
    )

# ==========================
# PATCH-BASED DEBLUR
# ==========================
def patch_deblur(frame):
    h, w, _ = frame.shape
    acc = np.zeros((h, w, 3), dtype=np.float32)
    weight = np.zeros((h, w, 1), dtype=np.float32)

    for y in range(0, h - TILE_SIZE + 1, STEP):
        for x in range(0, w - TILE_SIZE + 1, STEP):
            tile = frame[y:y+TILE_SIZE, x:x+TILE_SIZE]

            inp = (tile.astype(np.float32) / 255.0)
            inp = inp.transpose(2, 0, 1)[None]

            out = deb_sess.run(None, {deb_in: inp})[0][0]
            out = out.transpose(1, 2, 0)

            acc[y:y+TILE_SIZE, x:x+TILE_SIZE] += out
            weight[y:y+TILE_SIZE, x:x+TILE_SIZE] += 1.0

    deblurred = acc / np.maximum(weight, 1e-6)
    return (np.clip(deblurred, 0, 1) * 255).astype(np.uint8)

# ==========================
# OCR TEMPORAL BUFFER
# ==========================
buffer = deque(maxlen=TEMPORAL_WINDOW)
final_text = ""

# ==========================
# MAIN LOOP (WITH tqdm)
# ==========================
frame_id = 0
processed = 0

pbar = tqdm(total=total_to_process, desc="Processing frames", unit="frame")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    if frame_id % FRAME_SKIP != 0:
        frame_id += 1
        continue

    original = frame.copy()

    # -------------------------
    # PATCH-BASED DEBLUR
    # -------------------------
    deblurred = patch_deblur(frame)

    # -------------------------
    # OCR (RAW FRAME ONLY)
    # -------------------------
    if processed % OCR_EVERY_N == 0:
        x1, y1, x2, y2 = get_text_roi(original)
        roi = original[y1:y2, x1:x2]
        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)

        if has_text_like_content(gray):
            gray = cv2.resize(gray, (128, 32))
            ocr_inp = (gray.astype(np.float32) / 255.0)[None, None]

            logits = ocr_sess.run(None, {ocr_in: ocr_inp})[0]
            text, conf = decode_ctc(logits)

            if conf >= OCR_CONF_THRESH:
                buffer.append(text)
                if len(buffer) == TEMPORAL_WINDOW and len(set(buffer)) == 1:
                    final_text = buffer[0]
                    tqdm.write(f"[OCR ✓] {final_text}")

    # -------------------------
    # VISUALIZATION
    # -------------------------
    vis = deblurred.copy()
    if final_text:
        cv2.putText(
            vis,
            final_text,
            (30, 50),
            cv2.FONT_HERSHEY_SIMPLEX,
            1.3,
            (0, 255, 0),
            3
        )

    combined = np.hstack([original, vis])
    writer.write(combined)

    processed += 1
    frame_id += 1
    pbar.update(1)

pbar.close()
cap.release()
writer.release()

print("✅ DONE")
print(f"📽 Frames processed: {processed}")
print(f"🎬 Output saved to: {OUTPUT_PATH}")


In [ ]:
import cv2
import numpy as np
import onnxruntime as ort
from pathlib import Path
from collections import deque
from tqdm import tqdm

# ==========================
# CONFIG
# ==========================
VIDEO_PATH = Path("../data/raw_videos/high_speed_axis_55kmph_night.mp4")
OUTPUT_PATH = Path("patch_deblur_output.mp4")

DEBLUR_ONNX = "../onnx_models/deblur.onnx"
OCR_ONNX    = "../onnx_models/ocr.onnx"

FRAME_SKIP   = 5
OCR_EVERY_N  = 20
DEFAULT_FPS  = 50

# Patch parameters
TILE_SIZE = 256
OVERLAP   = 32
STEP      = 224

# OCR parameters
EDGE_VAR_THRESH = 80.0
OCR_CONF_THRESH = 0.65
TEMPORAL_WINDOW = 5

ALPHABET = "0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"
PROVIDERS = ["CUDAExecutionProvider", "CPUExecutionProvider"]

# ==========================
# LOAD MODELS
# ==========================
deb_sess = ort.InferenceSession(DEBLUR_ONNX, providers=PROVIDERS)
ocr_sess = ort.InferenceSession(OCR_ONNX, providers=PROVIDERS)

deb_in = deb_sess.get_inputs()[0].name
ocr_in = ocr_sess.get_inputs()[0].name

# ==========================
# VIDEO SETUP
# ==========================
cap = cv2.VideoCapture(str(VIDEO_PATH))
assert cap.isOpened(), "❌ Cannot open video"

total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
total_to_process = total_frames // FRAME_SKIP

fps = cap.get(cv2.CAP_PROP_FPS)
fps = fps if fps > 1 else DEFAULT_FPS
fps /= FRAME_SKIP

ret, first = cap.read()
assert ret, "❌ Empty video"

H, W = first.shape[:2]

writer = cv2.VideoWriter(
    str(OUTPUT_PATH),
    cv2.VideoWriter_fourcc(*"mp4v"),
    fps,
    (W * 2, H)
)

cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

# ==========================
# HELPERS
# ==========================
def softmax(x, axis=-1):
    x = x - np.max(x, axis=axis, keepdims=True)
    e = np.exp(x)
    return e / np.sum(e, axis=axis, keepdims=True)

def decode_ctc(logits):
    probs = softmax(logits, axis=-1)
    preds = probs.argmax(axis=-1)[0]

    prev = -1
    text, confs = "", []

    for t, p in enumerate(preds):
        if p != prev and p != 0:
            text += ALPHABET[p - 1]
            confs.append(probs[0, t, p])
        prev = p

    if not confs:
        return "", 0.0

    return text, float(np.mean(confs))

def has_text_like_content(gray):
    return cv2.Laplacian(gray, cv2.CV_64F).var() > EDGE_VAR_THRESH

def get_text_roi(frame):
    h, w = frame.shape[:2]
    return (
        int(w * 0.25),
        int(h * 0.45),
        int(w * 0.75),
        int(h * 0.65),
    )

# ==========================
# PATCH-BASED DEBLUR
# ==========================
def patch_deblur(frame):
    h, w, _ = frame.shape
    acc = np.zeros((h, w, 3), dtype=np.float32)
    weight = np.zeros((h, w, 1), dtype=np.float32)

    for y in range(0, h - TILE_SIZE + 1, STEP):
        for x in range(0, w - TILE_SIZE + 1, STEP):
            tile = frame[y:y+TILE_SIZE, x:x+TILE_SIZE]

            inp = (tile.astype(np.float32) / 255.0)
            inp = inp.transpose(2, 0, 1)[None]

            out = deb_sess.run(None, {deb_in: inp})[0][0]
            out = out.transpose(1, 2, 0)

            acc[y:y+TILE_SIZE, x:x+TILE_SIZE] += out
            weight[y:y+TILE_SIZE, x:x+TILE_SIZE] += 1.0

    deblurred = acc / np.maximum(weight, 1e-6)
    return (np.clip(deblurred, 0, 1) * 255).astype(np.uint8)

# ==========================
# OCR TEMPORAL BUFFER
# ==========================
buffer = deque(maxlen=TEMPORAL_WINDOW)
final_text = ""

# ==========================
# MAIN LOOP (WITH tqdm)
# ==========================
frame_id = 0
processed = 0

pbar = tqdm(total=total_to_process, desc="Processing frames", unit="frame")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    if frame_id % FRAME_SKIP != 0:
        frame_id += 1
        continue

    original = frame.copy()

    # -------------------------
    # PATCH-BASED DEBLUR
    # -------------------------
    deblurred = patch_deblur(frame)

    # -------------------------
    # OCR (RAW FRAME ONLY)
    # -------------------------
    if processed % OCR_EVERY_N == 0:
        x1, y1, x2, y2 = get_text_roi(original)
        roi = original[y1:y2, x1:x2]
        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)

        if has_text_like_content(gray):
            gray = cv2.resize(gray, (128, 32))
            ocr_inp = (gray.astype(np.float32) / 255.0)[None, None]

            logits = ocr_sess.run(None, {ocr_in: ocr_inp})[0]
            text, conf = decode_ctc(logits)

            if conf >= OCR_CONF_THRESH:
                buffer.append(text)
                if len(buffer) == TEMPORAL_WINDOW and len(set(buffer)) == 1:
                    final_text = buffer[0]
                    tqdm.write(f"[OCR ✓] {final_text}")

    # -------------------------
    # VISUALIZATION
    # -------------------------
    vis = deblurred.copy()
    if final_text:
        cv2.putText(
            vis,
            final_text,
            (30, 50),
            cv2.FONT_HERSHEY_SIMPLEX,
            1.3,
            (0, 255, 0),
            3
        )

    combined = np.hstack([original, vis])
    writer.write(combined)

    processed += 1
    frame_id += 1
    pbar.update(1)

pbar.close()
cap.release()
writer.release()

print("✅ DONE")
print(f"📽 Frames processed: {processed}")
print(f"🎬 Output saved to: {OUTPUT_PATH}")


In [ ]:
import cv2
import numpy as np
from tqdm import tqdm
from collections import defaultdict, deque
from pathlib import Path
import onnxruntime as ort
import pytesseract
import os

# =====================================================
# PATHS (ROBUST)
# =====================================================
VIDEO_PATH = Path("../data/raw_videos/high_speed_axis_55kmph_night.mp4")

LOWLIGHT_ONNX = Path("../onnx_models/lowlight.onnx")
DEBLUR_ONNX   = Path("../onnx_models/deblur.onnx")
OCR_ONNX      = Path("../onnx_models/ocr.onnx")

# ---- sanity checks ----
print("CWD:", os.getcwd())
print("Video resolved to:", VIDEO_PATH.resolve())
assert VIDEO_PATH.exists(), f"❌ Video not found: {VIDEO_PATH.resolve()}"
assert LOWLIGHT_ONNX.exists(), "❌ Missing lowlight ONNX"
assert DEBLUR_ONNX.exists(), "❌ Missing deblur ONNX"
assert OCR_ONNX.exists(), "❌ Missing OCR ONNX"

# =====================================================
# GAP DETECTION (ROBUST)
# =====================================================
MIN_GAP_DROP = 0.08          # normalized (night-safe)
COOLDOWN_FRAMES = 10

# =====================================================
# STRICT ENHANCEMENT THRESHOLDS
# =====================================================
LOWLIGHT_MEAN_THRESH = 40
LOWLIGHT_STD_THRESH  = 25
BLUR_LAPLACIAN_THRESH = 35.0

# =====================================================
# OCR CONFIG
# =====================================================
ALPHABET = "0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ"

FRAME_SKIP_OCR = 3
TEMPORAL_WINDOW = 5

EDGE_DENSITY_THRESH = 0.02
TESS_CONF_THRESH = 70

TESS_CONFIG = (
    "--oem 1 "
    "--psm 7 "
    "-c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"
)

# ROI (tune ONCE if needed)
ROI_X1_FRAC = 0.25
ROI_X2_FRAC = 0.75
ROI_Y1_FRAC = 0.45
ROI_Y2_FRAC = 0.65

# =====================================================
# PATCH DEBLUR PARAMS
# =====================================================
TILE_SIZE = 256
OVERLAP = 32
STEP = TILE_SIZE - OVERLAP

# =====================================================
# LOAD MODELS
# =====================================================
providers = ["CUDAExecutionProvider", "CPUExecutionProvider"]

low_sess = ort.InferenceSession(str(LOWLIGHT_ONNX), providers=providers)
deb_sess = ort.InferenceSession(str(DEBLUR_ONNX), providers=providers)
ocr_sess = ort.InferenceSession(str(OCR_ONNX), providers=providers)

low_in = low_sess.get_inputs()[0].name
deb_in = deb_sess.get_inputs()[0].name
ocr_in = ocr_sess.get_inputs()[0].name

# =====================================================
# HELPERS
# =====================================================
def softmax(x, axis=-1):
    x = x - np.max(x, axis=axis, keepdims=True)
    e = np.exp(x)
    return e / np.sum(e, axis=axis, keepdims=True)

def decode_ctc(logits):
    probs = softmax(logits, axis=-1)
    preds = probs.argmax(axis=-1)[0]
    prev = -1
    text = ""
    for p in preds:
        if p != prev and 1 <= p <= len(ALPHABET):
            text += ALPHABET[p - 1]
        prev = p
    return text

def is_text_like(gray):
    edges = cv2.Canny(gray, 50, 150)
    return (edges.mean() / 255.0) > EDGE_DENSITY_THRESH

def preprocess_for_tesseract(roi):
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    gray = cv2.normalize(gray, None, 0, 255, cv2.NORM_MINMAX)
    bin_img = cv2.adaptiveThreshold(
        gray, 255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY,
        31, 5
    )
    return cv2.morphologyEx(bin_img, cv2.MORPH_OPEN, np.ones((2,2), np.uint8))

def tesseract_ocr(bin_img):
    data = pytesseract.image_to_data(
        bin_img,
        config=TESS_CONFIG,
        output_type=pytesseract.Output.DICT
    )
    texts, confs = [], []
    for txt, conf in zip(data["text"], data["conf"]):
        if txt.strip() and int(conf) > 0:
            texts.append(txt)
            confs.append(int(conf))
    if not texts:
        return "", 0.0
    return "".join(texts), float(np.mean(confs))

def is_low_light(gray):
    return gray.mean() < LOWLIGHT_MEAN_THRESH and gray.std() < LOWLIGHT_STD_THRESH

def is_blurry(gray):
    return cv2.Laplacian(gray, cv2.CV_64F).var() < BLUR_LAPLACIAN_THRESH

def apply_zero_dce(img, curves):
    out = img
    for i in range(8):
        r = curves[:, i*3:(i+1)*3]
        out = out + r * (out*out - out)
    return np.clip(out, 0, 1)

def patch_deblur(frame):
    h, w, _ = frame.shape
    tiles, coords = [], []
    for y in range(0, h - TILE_SIZE + 1, STEP):
        for x in range(0, w - TILE_SIZE + 1, STEP):
            tiles.append(frame[y:y+TILE_SIZE, x:x+TILE_SIZE])
            coords.append((y, x))
    if not tiles:
        return frame

    batch = np.stack([
        t.astype(np.float32).transpose(2,0,1) / 255.0
        for t in tiles
    ])
    out = deb_sess.run(None, {deb_in: batch})[0].transpose(0,2,3,1)

    acc = np.zeros((h,w,3), np.float32)
    weight = np.zeros((h,w,1), np.float32)
    for (y,x), tile in zip(coords, out):
        acc[y:y+TILE_SIZE, x:x+TILE_SIZE] += tile
        weight[y:y+TILE_SIZE, x:x+TILE_SIZE] += 1

    return (np.clip(acc / np.maximum(weight,1e-6),0,1)*255).astype(np.uint8)

# =====================================================
# MAIN LOOP (SAFE)
# =====================================================
cap = cv2.VideoCapture(str(VIDEO_PATH))
assert cap.isOpened(), "❌ OpenCV cannot open the video"

prev_profile = None
cooldown = 0
wagon_index = 1

ocr_buffer = deque(maxlen=TEMPORAL_WINDOW)
wagon_texts = defaultdict(list)

print("🚆 START → Wagon #1")

pbar = tqdm(desc="Processing video", unit="frame")
frame_id = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_id += 1
    pbar.update(1)

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    profile = gray.mean(axis=0)

    # ---------- GAP DETECTION ----------
    if prev_profile is not None:
        diff = np.abs(prev_profile - profile)
        drop = diff.mean() / (profile.mean() + 1e-6)
        if drop > MIN_GAP_DROP and cooldown == 0:
            wagon_index += 1
            cooldown = COOLDOWN_FRAMES
            print(f"\n🔹 GAP → Wagon #{wagon_index}")

    prev_profile = profile
    cooldown = max(0, cooldown - 1)

    # ---------- STRICT VISUAL ENHANCEMENT ----------
    vis = frame.copy()
    if is_low_light(gray):
        rgb = cv2.cvtColor(vis, cv2.COLOR_BGR2RGB)
        inp = (rgb/255.0).transpose(2,0,1)[None]
        curves = low_sess.run(None, {low_in: inp})[0]
        vis = (apply_zero_dce(inp, curves)[0].transpose(1,2,0)*255).astype(np.uint8)

    if is_blurry(gray):
        vis = patch_deblur(vis)

    # ---------- OCR (RAW ONLY) ----------
    if frame_id % FRAME_SKIP_OCR == 0:
        H,W = frame.shape[:2]
        roi = frame[
            int(H*ROI_Y1_FRAC):int(H*ROI_Y2_FRAC),
            int(W*ROI_X1_FRAC):int(W*ROI_X2_FRAC)
        ]
        gray_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)

        if is_text_like(gray_roi):
            bin_img = preprocess_for_tesseract(roi)
            text, conf = tesseract_ocr(bin_img)

            if conf >= TESS_CONF_THRESH:
                ocr_buffer.append(text)
            else:
                resized = cv2.resize(gray_roi,(128,32))
                inp = (resized/255.0).astype(np.float32)[None,None]
                logits = ocr_sess.run(None,{ocr_in:inp})[0]
                ocr_buffer.append(decode_ctc(logits))

            if len(ocr_buffer)==TEMPORAL_WINDOW and len(set(ocr_buffer))==1:
                wagon_texts[wagon_index].append(ocr_buffer[0])
                print(f"[OCR] Wagon #{wagon_index}: {ocr_buffer[0]}")

cap.release()
pbar.close()

# =====================================================
# RESULTS
# =====================================================
print("\n==============================")
print(f"✅ TOTAL WAGONS: {wagon_index}")
print("==============================")

for i in range(1, wagon_index+1):
    texts = wagon_texts[i]
    if texts:
        print(f"Wagon #{i}: {max(set(texts), key=texts.count)}")
    else:
        print(f"Wagon #{i}: <NO TEXT>")


In [ ]:
import cv2
import numpy as np
from tqdm import tqdm
from collections import defaultdict, deque
import onnxruntime as ort
import pytesseract

# =====================================================
# PATHS
# =====================================================
VIDEO_PATH = "raw_videos/high_speed_axis_55kmph_night.mp4"

LOWLIGHT_ONNX = "../onnx_models/lowlight.onnx"
DEBLUR_ONNX   = "../onnx_models/deblur.onnx"
OCR_ONNX      = "../onnx_models/ocr.onnx"

# =====================================================
# GAP DETECTION (ROBUST)
# =====================================================
MIN_GAP_DROP = 0.08          # normalized drop (night-safe)
COOLDOWN_FRAMES = 10

# =====================================================
# STRICT ENHANCEMENT THRESHOLDS
# =====================================================
LOWLIGHT_MEAN_THRESH = 40
LOWLIGHT_STD_THRESH  = 25
BLUR_LAPLACIAN_THRESH = 35.0

# =====================================================
# OCR CONFIG
# =====================================================
ALPHABET = "0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ"

FRAME_SKIP_OCR = 3
TEMPORAL_WINDOW = 5

EDGE_DENSITY_THRESH = 0.02
TESS_CONF_THRESH = 70

TESS_CONFIG = (
    "--oem 1 "
    "--psm 7 "
    "-c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"
)

# ROI (tune ONCE if needed)
ROI_X1_FRAC = 0.25
ROI_X2_FRAC = 0.75
ROI_Y1_FRAC = 0.45
ROI_Y2_FRAC = 0.65

# =====================================================
# PATCH DEBLUR PARAMS
# =====================================================
TILE_SIZE = 256
OVERLAP = 32
STEP = TILE_SIZE - OVERLAP

# =====================================================
# LOAD MODELS
# =====================================================
providers = ["CUDAExecutionProvider", "CPUExecutionProvider"]

low_sess = ort.InferenceSession(LOWLIGHT_ONNX, providers=providers)
deb_sess = ort.InferenceSession(DEBLUR_ONNX, providers=providers)
ocr_sess = ort.InferenceSession(OCR_ONNX, providers=providers)

low_in = low_sess.get_inputs()[0].name
deb_in = deb_sess.get_inputs()[0].name
ocr_in = ocr_sess.get_inputs()[0].name

# =====================================================
# HELPERS
# =====================================================
def softmax(x, axis=-1):
    x = x - np.max(x, axis=axis, keepdims=True)
    e = np.exp(x)
    return e / np.sum(e, axis=axis, keepdims=True)

def decode_ctc(logits):
    probs = softmax(logits, axis=-1)
    preds = probs.argmax(axis=-1)[0]
    prev = -1
    text = ""
    for p in preds:
        if p != prev and 1 <= p <= len(ALPHABET):
            text += ALPHABET[p - 1]
        prev = p
    return text

def is_text_like(gray):
    edges = cv2.Canny(gray, 50, 150)
    return (edges.mean() / 255.0) > EDGE_DENSITY_THRESH

def preprocess_for_tesseract(roi):
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    gray = cv2.normalize(gray, None, 0, 255, cv2.NORM_MINMAX)
    bin_img = cv2.adaptiveThreshold(
        gray, 255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY,
        31, 5
    )
    return cv2.morphologyEx(bin_img, cv2.MORPH_OPEN, np.ones((2,2), np.uint8))

def tesseract_ocr(bin_img):
    data = pytesseract.image_to_data(
        bin_img,
        config=TESS_CONFIG,
        output_type=pytesseract.Output.DICT
    )
    texts, confs = [], []
    for txt, conf in zip(data["text"], data["conf"]):
        if txt.strip() and int(conf) > 0:
            texts.append(txt)
            confs.append(int(conf))
    if not texts:
        return "", 0.0
    return "".join(texts), float(np.mean(confs))

def is_low_light(gray):
    return gray.mean() < LOWLIGHT_MEAN_THRESH and gray.std() < LOWLIGHT_STD_THRESH

def is_blurry(gray):
    return cv2.Laplacian(gray, cv2.CV_64F).var() < BLUR_LAPLACIAN_THRESH

def apply_zero_dce(img, curves):
    out = img
    for i in range(8):
        r = curves[:, i*3:(i+1)*3]
        out = out + r * (out*out - out)
    return np.clip(out, 0, 1)

def patch_deblur(frame):
    h, w, _ = frame.shape
    tiles, coords = [], []
    for y in range(0, h - TILE_SIZE + 1, STEP):
        for x in range(0, w - TILE_SIZE + 1, STEP):
            tiles.append(frame[y:y+TILE_SIZE, x:x+TILE_SIZE])
            coords.append((y, x))
    if not tiles:
        return frame
    batch = np.stack([
        t.astype(np.float32).transpose(2,0,1) / 255.0
        for t in tiles
    ])
    out = deb_sess.run(None, {deb_in: batch})[0].transpose(0,2,3,1)
    acc = np.zeros((h,w,3), np.float32)
    weight = np.zeros((h,w,1), np.float32)
    for (y,x), tile in zip(coords, out):
        acc[y:y+TILE_SIZE, x:x+TILE_SIZE] += tile
        weight[y:y+TILE_SIZE, x:x+TILE_SIZE] += 1
    return (np.clip(acc / np.maximum(weight,1e-6),0,1)*255).astype(np.uint8)

# =====================================================
# MAIN LOOP (FIXED)
# =====================================================
cap = cv2.VideoCapture(VIDEO_PATH)
assert cap.isOpened(), f"❌ Cannot open video: {VIDEO_PATH}"

prev_profile = None
cooldown = 0
wagon_index = 0

ocr_buffer = deque(maxlen=TEMPORAL_WINDOW)
wagon_texts = defaultdict(list)

print("🚆 START → Wagon #1")

pbar = tqdm(desc="Processing video", unit="frame")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    pbar.update(1)

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    profile = gray.mean(axis=0)

    # ---------- GAP DETECTION ----------
    if prev_profile is not None:
        diff = np.abs(prev_profile - profile)
        drop = diff.mean() / (profile.mean() + 1e-6)
        if drop > MIN_GAP_DROP and cooldown == 0:
            wagon_index += 1
            cooldown = COOLDOWN_FRAMES
            print(f"\n🔹 GAP → Wagon #{wagon_index}")

    prev_profile = profile
    cooldown = max(0, cooldown - 1)

    # ---------- STRICT VISUAL ENHANCEMENT ----------
    vis = frame.copy()
    if is_low_light(gray):
        rgb = cv2.cvtColor(vis, cv2.COLOR_BGR2RGB)
        inp = (rgb/255.0).transpose(2,0,1)[None]
        curves = low_sess.run(None, {low_in: inp})[0]
        vis = (apply_zero_dce(inp, curves)[0].transpose(1,2,0)*255).astype(np.uint8)

    if is_blurry(gray):
        vis = patch_deblur(vis)

    # ---------- OCR (RAW ONLY) ----------
    if frame_id = pbar.n
    if wagon_index >= 1 and frame_id % FRAME_SKIP_OCR == 0:
        H,W = frame.shape[:2]
        roi = frame[
            int(H*ROI_Y1_FRAC):int(H*ROI_Y2_FRAC),
            int(W*ROI_X1_FRAC):int(W*ROI_X2_FRAC)
        ]
        gray_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        if is_text_like(gray_roi):
            bin_img = preprocess_for_tesseract(roi)
            text, conf = tesseract_ocr(bin_img)
            if conf >= TESS_CONF_THRESH:
                ocr_buffer.append(text)
            else:
                resized = cv2.resize(gray_roi,(128,32))
                inp = (resized/255.0).astype(np.float32)[None,None]
                logits = ocr_sess.run(None,{ocr_in:inp})[0]
                ocr_buffer.append(decode_ctc(logits))
            if len(ocr_buffer)==TEMPORAL_WINDOW and len(set(ocr_buffer))==1:
                wagon_texts[wagon_index].append(ocr_buffer[0])
                print(f"[OCR] Wagon #{wagon_index}: {ocr_buffer[0]}")

cap.release()

# =====================================================
# RESULTS
# =====================================================
print("\n==============================")
print(f"✅ TOTAL WAGONS: {wagon_index}")
print("==============================")
for i in range(1, wagon_index+1):
    texts = wagon_texts[i]
    if texts:
        print(f"Wagon #{i}: {max(set(texts), key=texts.count)}")
    else:
        print(f"Wagon #{i}: <NO TEXT>")


In [ ]:
from pathlib import Path

VIDEO_PATH = Path("raw_videos/high_speed_axis_55kmph_night.mp4")
print("Exists:", VIDEO_PATH.exists())
print("Absolute path:", VIDEO_PATH.resolve())


In [ ]:
import cv2
import numpy as np
from tqdm import tqdm
from collections import defaultdict, deque
from pathlib import Path
import onnxruntime as ort
import os

# =====================================================
# PATHS
# =====================================================
VIDEO_PATH = Path("../data/raw_videos/high_speed_axis_55kmph_night.mp4")

LOWLIGHT_ONNX = Path("../onnx_models/lowlight.onnx")
DEBLUR_ONNX   = Path("../onnx_models/deblur.onnx")
OCR_ONNX      = Path("../onnx_models/ocr.onnx")

print("CWD:", os.getcwd())
print("Video:", VIDEO_PATH.resolve())
assert VIDEO_PATH.exists(), "❌ Video not found"

# =====================================================
# GAP DETECTION
# =====================================================
MIN_GAP_DROP = 0.08
COOLDOWN_FRAMES = 10

# =====================================================
# STRICT ENHANCEMENT
# =====================================================
LOWLIGHT_MEAN_THRESH = 40
LOWLIGHT_STD_THRESH  = 25
BLUR_LAPLACIAN_THRESH = 35.0

# =====================================================
# OCR (CRNN ONLY)
# =====================================================
ALPHABET = "0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ"

FRAME_SKIP_OCR = 3
TEMPORAL_WINDOW = 5

EDGE_DENSITY_THRESH = 0.02
CRNN_CONF_THRESH = 0.75   # VERY IMPORTANT

ROI_X1_FRAC = 0.25
ROI_X2_FRAC = 0.75
ROI_Y1_FRAC = 0.45
ROI_Y2_FRAC = 0.65

# =====================================================
# PATCH DEBLUR PARAMS
# =====================================================
TILE_SIZE = 256
OVERLAP = 32
STEP = TILE_SIZE - OVERLAP

# =====================================================
# LOAD MODELS
# =====================================================
providers = ["CUDAExecutionProvider", "CPUExecutionProvider"]

low_sess = ort.InferenceSession(str(LOWLIGHT_ONNX), providers=providers)
deb_sess = ort.InferenceSession(str(DEBLUR_ONNX), providers=providers)
ocr_sess = ort.InferenceSession(str(OCR_ONNX), providers=providers)

low_in = low_sess.get_inputs()[0].name
deb_in = deb_sess.get_inputs()[0].name
ocr_in = ocr_sess.get_inputs()[0].name

# =====================================================
# HELPERS
# =====================================================
def softmax(x, axis=-1):
    x = x - np.max(x, axis=axis, keepdims=True)
    e = np.exp(x)
    return e / np.sum(e, axis=axis, keepdims=True)

def decode_ctc_with_conf(logits):
    probs = softmax(logits, axis=-1)
    preds = probs.argmax(axis=-1)[0]

    prev = -1
    text = ""
    confs = []

    for t, p in enumerate(preds):
        if p != prev and p > 0 and p <= len(ALPHABET):
            text += ALPHABET[p - 1]
            confs.append(probs[0, t, p])
        prev = p

    mean_conf = float(np.mean(confs)) if confs else 0.0
    return text, mean_conf

def is_text_like(gray):
    edges = cv2.Canny(gray, 50, 150)
    return (edges.mean() / 255.0) > EDGE_DENSITY_THRESH

def is_low_light(gray):
    return gray.mean() < LOWLIGHT_MEAN_THRESH and gray.std() < LOWLIGHT_STD_THRESH

def is_blurry(gray):
    return cv2.Laplacian(gray, cv2.CV_64F).var() < BLUR_LAPLACIAN_THRESH

def apply_zero_dce(img, curves):
    out = img
    for i in range(8):
        r = curves[:, i*3:(i+1)*3]
        out = out + r * (out*out - out)
    return np.clip(out, 0, 1)

def patch_deblur(frame):
    h, w, _ = frame.shape
    tiles, coords = [], []

    for y in range(0, h - TILE_SIZE + 1, STEP):
        for x in range(0, w - TILE_SIZE + 1, STEP):
            tiles.append(frame[y:y+TILE_SIZE, x:x+TILE_SIZE])
            coords.append((y, x))

    if not tiles:
        return frame

    batch = np.stack([
        t.astype(np.float32).transpose(2,0,1) / 255.0
        for t in tiles
    ])

    out = deb_sess.run(None, {deb_in: batch})[0].transpose(0,2,3,1)

    acc = np.zeros((h,w,3), np.float32)
    weight = np.zeros((h,w,1), np.float32)

    for (y,x), tile in zip(coords, out):
        acc[y:y+TILE_SIZE, x:x+TILE_SIZE] += tile
        weight[y:y+TILE_SIZE, x:x+TILE_SIZE] += 1

    return (np.clip(acc / np.maximum(weight,1e-6),0,1)*255).astype(np.uint8)

# =====================================================
# MAIN LOOP
# =====================================================
cap = cv2.VideoCapture(str(VIDEO_PATH))
assert cap.isOpened(), "❌ Cannot open video"

prev_profile = None
cooldown = 0
wagon_index = 1

ocr_buffer = deque(maxlen=TEMPORAL_WINDOW)
wagon_texts = defaultdict(list)

print("🚆 START → Wagon #1")

pbar = tqdm(desc="Processing video", unit="frame")
frame_id = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_id += 1
    pbar.update(1)

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    profile = gray.mean(axis=0)

    # ---------- GAP DETECTION ----------
    if prev_profile is not None:
        diff = np.abs(prev_profile - profile)
        drop = diff.mean() / (profile.mean() + 1e-6)
        if drop > MIN_GAP_DROP and cooldown == 0:
            wagon_index += 1
            cooldown = COOLDOWN_FRAMES
            print(f"\n🔹 GAP → Wagon #{wagon_index}")

    prev_profile = profile
    cooldown = max(0, cooldown - 1)

    # ---------- STRICT VISUAL ENHANCEMENT ----------
    vis = frame.copy()
    if is_low_light(gray):
        rgb = cv2.cvtColor(vis, cv2.COLOR_BGR2RGB)
        inp = (rgb/255.0).transpose(2,0,1)[None]
        curves = low_sess.run(None, {low_in: inp})[0]
        vis = (apply_zero_dce(inp, curves)[0].transpose(1,2,0)*255).astype(np.uint8)

    if is_blurry(gray):
        vis = patch_deblur(vis)

    # ---------- OCR (CRNN ONLY, RAW FRAME) ----------
    if frame_id % FRAME_SKIP_OCR == 0:
        H,W = frame.shape[:2]
        roi = frame[
            int(H*ROI_Y1_FRAC):int(H*ROI_Y2_FRAC),
            int(W*ROI_X1_FRAC):int(W*ROI_X2_FRAC)
        ]

        gray_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)

        if is_text_like(gray_roi):
            resized = cv2.resize(gray_roi, (128,32))
            inp = (resized/255.0).astype(np.float32)[None,None]

            logits = ocr_sess.run(None, {ocr_in: inp})[0]
            text, conf = decode_ctc_with_conf(logits)

            if conf >= CRNN_CONF_THRESH:
                ocr_buffer.append(text)

            if len(ocr_buffer) == TEMPORAL_WINDOW and len(set(ocr_buffer)) == 1:
                wagon_texts[wagon_index].append(ocr_buffer[0])
                print(f"[OCR] Wagon #{wagon_index}: {ocr_buffer[0]}")

cap.release()
pbar.close()

# =====================================================
# RESULTS
# =====================================================
print("\n==============================")
print(f"✅ TOTAL WAGONS: {wagon_index}")
print("==============================")

for i in range(1, wagon_index+1):
    texts = wagon_texts[i]
    if texts:
        print(f"Wagon #{i}: {max(set(texts), key=texts.count)}")
    else:
        print(f"Wagon #{i}: <NO TEXT>")


In [ ]:
import cv2
import numpy as np
from pathlib import Path
from tqdm import tqdm

# =====================
# CONFIG
# =====================
VIDEO_PATH = Path("../data/raw_videos/high_speed_axis_55kmph_night.mp4")
RESIZE_WIDTH = 960

LINE_X_RATIO = 0.5
STRIP_WIDTH = 12

ROI_Y1_RATIO = 0.55
ROI_Y2_RATIO = 0.75

# ---- FIXED THRESHOLDS ----
DIFF_NORM_THRESHOLD = 0.06     # normalized, night-safe
COOLDOWN_FRAMES = 40

NO_MOTION_FRAMES = 300         # ~4 seconds at ~75fps

# =====================
# INIT
# =====================
cap = cv2.VideoCapture(str(VIDEO_PATH))
assert cap.isOpened(), "❌ Cannot open video"

print("🚆 Wagon Counting Started")

wagon_count = 0
cooldown = 0
no_motion = 0

prev_strip = None
motion_active = False   # NEW: edge detection

pbar = tqdm(desc="Processing video", unit="frame")

# =====================
# MAIN LOOP
# =====================
while True:
    ret, frame = cap.read()
    if not ret:
        break

    pbar.update(1)

    h, w = frame.shape[:2]
    scale = RESIZE_WIDTH / w
    frame = cv2.resize(frame, (RESIZE_WIDTH, int(h * scale)))
    H, W = frame.shape[:2]

    x = int(W * LINE_X_RATIO)
    y1 = int(H * ROI_Y1_RATIO)
    y2 = int(H * ROI_Y2_RATIO)

    strip = frame[y1:y2, x-STRIP_WIDTH:x+STRIP_WIDTH]
    gray = cv2.cvtColor(strip, cv2.COLOR_BGR2GRAY)

    if prev_strip is None:
        prev_strip = gray
        continue

    diff = cv2.absdiff(gray, prev_strip)
    mean_diff = diff.mean() / (gray.mean() + 1e-6)
    prev_strip = gray

    # =====================
    # COUNTING LOGIC (FIXED)
    # =====================
    if mean_diff > DIFF_NORM_THRESHOLD:
        no_motion = 0

        if not motion_active and cooldown == 0:
            wagon_count += 1
            cooldown = COOLDOWN_FRAMES
            motion_active = True
            print(f"🆕 WAGON #{wagon_count} DETECTED")

    else:
        motion_active = False
        no_motion += 1

    if cooldown > 0:
        cooldown -= 1

    # Stop when train gone (FIXED)
    if no_motion > NO_MOTION_FRAMES and wagon_count > 0:
        print("\n🚫 Train exited scene. Counting stopped.")
        break

    # =====================
    # VISUALIZATION
    # =====================
    cv2.line(frame, (x, 0), (x, H), (0, 0, 255), 2)
    cv2.rectangle(frame, (x-STRIP_WIDTH, y1), (x+STRIP_WIDTH, y2), (255, 0, 0), 2)

    cv2.putText(
        frame,
        f"Wagons: {wagon_count}",
        (20, 40),
        cv2.FONT_HERSHEY_SIMPLEX,
        1,
        (255, 255, 255),
        2
    )

    cv2.imshow("Robust Wagon Counter", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# =====================
# DONE
# =====================
cap.release()
cv2.destroyAllWindows()
pbar.close()

print("\n===================================")
print(f"✅ TOTAL WAGONS COUNTED: {wagon_count}")
print("===================================")


In [5]:
import cv2
import numpy as np
import onnxruntime as ort
from pathlib import Path
from tqdm import tqdm
import csv

# =====================
# PATHS
# =====================
PROJECT_ROOT = Path(r"C:\Users\Manas Mehta\Desktop\PROJECTS\AIDTM")
VIDEO_PATH = PROJECT_ROOT / "data" / "raw_videos" / "high_speed_axis_55kmph_night.mp4"
OCR_ONNX = PROJECT_ROOT / "onnx_models" / "ocr.onnx"
CSV_OUT = PROJECT_ROOT / "wagon_results.csv"

print("📂 Project root:", PROJECT_ROOT)
print("🎥 Video:", VIDEO_PATH)

assert VIDEO_PATH.exists(), "❌ Video not found"

OCR_ENABLED = OCR_ONNX.exists()
print("🔠 OCR enabled" if OCR_ENABLED else "⚠ OCR disabled")

# =====================
# CONFIG
# =====================
RESIZE_WIDTH = 960
LINE_X_RATIO = 0.50
STRIP_WIDTH = 10

ROI_Y1_RATIO = 0.55
ROI_Y2_RATIO = 0.75

DIFF_THRESHOLD = 18
COOLDOWN_FRAMES = 30
NO_MOTION_FRAMES = 90

SHOW_VIDEO = True
OCR_CONF_THRESH = 0.5

# =====================
# OCR
# =====================
ALPHABET = "0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ"

def decode_safe(logits):
    probs = logits[0]
    seq = probs.argmax(axis=1)
    text, confs = "", []
    prev = -1
    for t, c in enumerate(seq):
        if c != prev and 0 < c <= len(ALPHABET):
            text += ALPHABET[c - 1]
            confs.append(probs[t, c])
        prev = c
    return text, float(np.mean(confs)) if confs else 0.0

sess = None
ocr_input = None
if OCR_ENABLED:
    sess = ort.InferenceSession(str(OCR_ONNX), providers=["CPUExecutionProvider"])
    ocr_input = sess.get_inputs()[0].name

# =====================
# VIDEO
# =====================
cap = cv2.VideoCapture(str(VIDEO_PATH))
assert cap.isOpened(), "❌ Cannot open video"

fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

print("🚆 Wagon counting started")

wagon_count = 0
cooldown = 0
no_motion = 0
train_seen = False

wagon_frames = []
wagon_times = []
wagon_texts = {}

prev_strip = None

# =====================
# MAIN LOOP
# =====================
for frame_idx in tqdm(range(total_frames), desc="Processing video"):
    ret, frame = cap.read()
    if not ret:
        break

    h, w = frame.shape[:2]
    scale = RESIZE_WIDTH / w
    frame = cv2.resize(frame, (RESIZE_WIDTH, int(h * scale)))
    H, W = frame.shape[:2]

    x = int(W * LINE_X_RATIO)
    y1 = int(H * ROI_Y1_RATIO)
    y2 = int(H * ROI_Y2_RATIO)

    strip = frame[y1:y2, x-STRIP_WIDTH:x+STRIP_WIDTH]
    gray = cv2.cvtColor(strip, cv2.COLOR_BGR2GRAY)

    if prev_strip is None:
        prev_strip = gray
        continue

    diff = cv2.absdiff(gray, prev_strip).mean()
    prev_strip = gray

    # ===== TRAIN LOGIC =====
    if diff > DIFF_THRESHOLD:
        train_seen = True
        no_motion = 0

        if cooldown == 0:
            wagon_count += 1
            wagon_frames.append(frame_idx)
            wagon_times.append(frame_idx / fps)
            print(f"🆕 WAGON #{wagon_count} at frame {frame_idx}")

            # OCR (non-blocking)
            if OCR_ENABLED:
                roi = frame[int(H*0.35):int(H*0.55), int(W*0.60):int(W*0.95)]
                roi_gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
                roi_gray = cv2.resize(roi_gray, (128, 32))
                inp = (roi_gray / 255.0).astype(np.float32)[None, None]
                if inp.shape == (1,1,32,128):
                    logits = sess.run(None, {ocr_input: inp})[0]
                    txt, conf = decode_safe(logits)
                    wagon_texts[wagon_count] = txt if conf >= OCR_CONF_THRESH else "<LOW_CONF>"
                else:
                    wagon_texts[wagon_count] = "<OCR_SHAPE_ERR>"
            else:
                wagon_texts[wagon_count] = "<OCR_DISABLED>"

            cooldown = COOLDOWN_FRAMES

    else:
        if train_seen:
            no_motion += 1

    if cooldown > 0:
        cooldown -= 1

    if train_seen and no_motion > NO_MOTION_FRAMES:
        print("🚫 Train exited scene")
        break

    # ===== VIS =====
    if SHOW_VIDEO:
        cv2.line(frame, (x,0), (x,H), (0,0,255), 2)
        cv2.rectangle(frame, (x-STRIP_WIDTH,y1), (x+STRIP_WIDTH,y2), (255,0,0), 2)
        cv2.putText(frame, f"Wagons: {wagon_count}", (20,40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
        cv2.imshow("Wagon Counter", frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

cap.release()
cv2.destroyAllWindows()

# =====================
# CSV
# =====================
with open(CSV_OUT, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["wagon_index", "frame", "timestamp_sec", "ocr_text"])
    for i in range(wagon_count):
        writer.writerow([
            i+1,
            wagon_frames[i],
            round(wagon_times[i], 3),
            wagon_texts.get(i+1, "")
        ])

print("\n===================================")
print(f"✅ TOTAL WAGONS: {wagon_count}")
print(f"📄 CSV saved to: {CSV_OUT}")
print("===================================")


📂 Project root: C:\Users\Manas Mehta\Desktop\PROJECTS\AIDTM
🎥 Video: C:\Users\Manas Mehta\Desktop\PROJECTS\AIDTM\data\raw_videos\high_speed_axis_55kmph_night.mp4
🔠 OCR enabled
🚆 Wagon counting started


Processing video:  36%|███▌      | 1100/3082 [00:10<00:21, 93.61it/s] 

🆕 WAGON #1 at frame 1089


Processing video:  37%|███▋      | 1132/3082 [00:11<00:20, 97.18it/s]

🆕 WAGON #2 at frame 1119


...

🆕 WAGON #57 at frame 2792


Processing video:  92%|█████████▏| 2829/3082 [00:34<00:03, 68.50it/s]

🆕 WAGON #58 at frame 2822


Processing video:  95%|█████████▍| 2918/3082 [00:36<00:02, 80.84it/s]

🚫 Train exited scene

✅ TOTAL WAGONS: 58
📄 CSV saved to: C:\Users\Manas Mehta\Desktop\PROJECTS\AIDTM\wagon_results.csv


In [9]:
# ============================================================
# FULL WAGON COUNTING + FIXED OCR (SINGLE JUPYTER CELL)
# ============================================================

import cv2
import numpy as np
import csv
from pathlib import Path
from tqdm import tqdm
import onnxruntime as ort

# ============================================================
# PATHS (EDIT ONLY IF NEEDED)
# ============================================================
VIDEO_PATH = Path(r"C:\Users\Manas Mehta\Desktop\PROJECTS\AIDTM\data\raw_videos\high_speed_axis_55kmph_night.mp4")
OCR_ONNX  = Path(r"C:\Users\Manas Mehta\Desktop\PROJECTS\AIDTM\onnx_models\ocr.onnx")

print("📂 Project root:", VIDEO_PATH.parents[2])
print("🎥 Video:", VIDEO_PATH)
print("🔠 OCR model:", OCR_ONNX)

if not VIDEO_PATH.exists():
    raise FileNotFoundError(f"❌ Video not found: {VIDEO_PATH}")
if not OCR_ONNX.exists():
    raise FileNotFoundError(f"❌ OCR model not found: {OCR_ONNX}")

# ============================================================
# OCR SETUP
# ============================================================
ALPHABET = "0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ"

def preprocess_for_ocr(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # CLAHE for night contrast
    clahe = cv2.createCLAHE(3.0, (8,8))
    gray = clahe.apply(gray)

    # Adaptive threshold (CRITICAL)
    th = cv2.adaptiveThreshold(
        gray, 255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY_INV,
        31, 15
    )

    # Morphological cleanup
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
    th = cv2.morphologyEx(th, cv2.MORPH_CLOSE, kernel)

    return th


def safe_decode(logits):
    probs = logits[0]
    seq = probs.argmax(axis=1)

    txt = ""
    confs = []
    prev = -1

    for t, c in enumerate(seq):
        if c != prev and c > 0 and c-1 < len(ALPHABET):
            txt += ALPHABET[c-1]
            confs.append(probs[t, c])
        prev = c

    if not confs:
        return "", 0.0
    return txt, float(np.mean(confs))


print("🔠 OCR enabled")
sess = ort.InferenceSession(str(OCR_ONNX), providers=["CPUExecutionProvider"])
ocr_input = sess.get_inputs()[0].name

# ============================================================
# WAGON COUNTING CONFIG (UNCHANGED LOGIC)
# ============================================================
RESIZE_WIDTH = 960
LINE_X_RATIO = 0.5
STRIP_WIDTH = 10

ROI_Y1_RATIO = 0.55
ROI_Y2_RATIO = 0.75

DIFF_THRESHOLD = 18
COOLDOWN_FRAMES = 30
NO_MOTION_FRAMES = 60

# ============================================================
# VIDEO INIT
# ============================================================
cap = cv2.VideoCapture(str(VIDEO_PATH))
assert cap.isOpened(), "❌ Cannot open video"

fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

wagon_count = 0
cooldown = 0
no_motion = 0
prev_strip = None
train_started = False

results = []

print("🚆 Wagon counting started")

# ============================================================
# MAIN LOOP
# ============================================================
for frame_idx in tqdm(range(total_frames)):
    ret, frame = cap.read()
    if not ret:
        break

    h, w = frame.shape[:2]
    scale = RESIZE_WIDTH / w
    frame = cv2.resize(frame, (RESIZE_WIDTH, int(h * scale)))
    H, W = frame.shape[:2]

    x = int(W * LINE_X_RATIO)
    y1 = int(H * ROI_Y1_RATIO)
    y2 = int(H * ROI_Y2_RATIO)

    strip = frame[y1:y2, x-STRIP_WIDTH:x+STRIP_WIDTH]
    gray_strip = cv2.cvtColor(strip, cv2.COLOR_BGR2GRAY)

    if prev_strip is None:
        prev_strip = gray_strip
        continue

    diff = cv2.absdiff(gray_strip, prev_strip)
    mean_diff = diff.mean()
    prev_strip = gray_strip

    if mean_diff > DIFF_THRESHOLD:
        no_motion = 0

        if not train_started:
            train_started = True
            print(f"🚆 Train confirmed at frame {frame_idx}")

        if cooldown == 0:
            wagon_count += 1
            cooldown = COOLDOWN_FRAMES
            ts = frame_idx / fps

            print(f"🆕 WAGON #{wagon_count} at frame {frame_idx}")

            # =============================
            # FIXED OCR (MULTI-FRAME VOTING)
            # =============================
            texts = []
            for offset in [5, 8, 12]:
                cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx + offset)
                r, f2 = cap.read()
                if not r:
                    continue

                roi = f2[y1:y2, x+40:x+300]
                roi = cv2.resize(roi, (128, 32))
                th = preprocess_for_ocr(roi)

                inp = (th / 255.0).astype(np.float32)[None, None]
                logits = sess.run(None, {ocr_input: inp})[0]
                txt, conf = safe_decode(logits)

                if conf > 0.55 and len(txt) >= 3:
                    texts.append((txt, conf))

            best_text = max(texts, key=lambda x: x[1])[0] if texts else "<NO_TEXT>"

            results.append([wagon_count, frame_idx, round(ts,3), best_text])

    else:
        if train_started:
            no_motion += 1

    if cooldown > 0:
        cooldown -= 1

    if train_started and no_motion > NO_MOTION_FRAMES:
        print("🚫 Train exited scene")
        break

    # =============================
    # VISUAL DISPLAY
    # =============================
    cv2.line(frame, (x,0), (x,H), (0,0,255), 2)
    cv2.rectangle(frame, (x-STRIP_WIDTH,y1), (x+STRIP_WIDTH,y2), (255,0,0), 2)
    cv2.putText(frame, f"Wagons: {wagon_count}", (20,40),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)

    cv2.imshow("Wagon Counter + OCR", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# ============================================================
# SAVE CSV
# ============================================================
out_csv = VIDEO_PATH.parents[1] / "wagon_results.csv"

with open(out_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["wagon_index", "frame", "timestamp_sec", "ocr_text"])
    writer.writerows(results)

print("\n===================================")
print(f"✅ TOTAL WAGONS: {len(results)}")
print(f"📄 CSV saved to: {out_csv}")
print("===================================")


📂 Project root: C:\Users\Manas Mehta\Desktop\PROJECTS\AIDTM
🎥 Video: C:\Users\Manas Mehta\Desktop\PROJECTS\AIDTM\data\raw_videos\high_speed_axis_55kmph_night.mp4
🔠 OCR model: C:\Users\Manas Mehta\Desktop\PROJECTS\AIDTM\onnx_models\ocr.onnx
🔠 OCR enabled
🚆 Wagon counting started


 35%|███▌      | 1088/3082 [00:11<00:19, 100.64it/s]

🚆 Train confirmed at frame 1089
🆕 WAGON #1 at frame 1089


 36%|███▋      | 1119/3082 [00:12<00:29, 66.59it/s] 

🆕 WAGON #2 at frame 1120


...

🆕 WAGON #56 at frame 2760


 91%|█████████ | 2790/3082 [00:53<00:05, 52.39it/s]

🆕 WAGON #57 at frame 2790


 93%|█████████▎| 2875/3082 [00:55<00:03, 52.09it/s]

🚫 Train exited scene

✅ TOTAL WAGONS: 57
📄 CSV saved to: C:\Users\Manas Mehta\Desktop\PROJECTS\AIDTM\data\wagon_results.csv


In [ ]:
from pathlib import Path

# =====================================================
# JUPYTER-SAFE PROJECT ROOT RESOLUTION
# =====================================================
NOTEBOOK_DIR = Path.cwd()          # .../AIDTM/notebooks
PROJECT_ROOT = NOTEBOOK_DIR.parent # .../AIDTM

VIDEO_PATH = PROJECT_ROOT / "data" / "raw_videos" / "high_speed_axis_55kmph_night.mp4"
OCR_ONNX   = PROJECT_ROOT / "onnx_models" / "ocr.onnx"

print("📂 Notebook dir:", NOTEBOOK_DIR)
print("📂 Project root:", PROJECT_ROOT)
print("🎥 Video:", VIDEO_PATH)
print("🔠 OCR model:", OCR_ONNX)

assert VIDEO_PATH.exists(), f"❌ Video not found: {VIDEO_PATH}"
assert OCR_ONNX.exists(), f"❌ OCR model not found: {OCR_ONNX}"


📂 Project root: c:\Users\Manas Mehta\Desktop\PROJECTS\AIDTM\notebooks
🎥 Video: c:\Users\Manas Mehta\Desktop\PROJECTS\AIDTM\notebooks\data\raw_videos\high_speed_axis_55kmph_night.mp4
🔠 OCR model: c:\Users\Manas Mehta\Desktop\PROJECTS\AIDTM\notebooks\onnx_models\ocr.onnx


NoSuchFile: [ONNXRuntimeError] : 3 : NO_SUCHFILE : Load model from c:\Users\Manas Mehta\Desktop\PROJECTS\AIDTM\notebooks\onnx_models\ocr.onnx failed:Load model c:\Users\Manas Mehta\Desktop\PROJECTS\AIDTM\notebooks\onnx_models\ocr.onnx failed. File doesn't exist

In [12]:
# =====================================================
# FULL STABLE WAGON COUNTING + FIXED OCR (ONE CELL)
# JUPYTER SAFE | NO TESSERACT | OCR ONNX
# =====================================================

import cv2
import numpy as np
import csv
from pathlib import Path
from collections import defaultdict, Counter
from tqdm import tqdm
import onnxruntime as ort

# =====================================================
# PATHS (JUPYTER SAFE)
# =====================================================
NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = NOTEBOOK_DIR.parent

VIDEO_PATH = PROJECT_ROOT / "data" / "raw_videos" / "high_speed_axis_55kmph_night.mp4"
OCR_ONNX   = PROJECT_ROOT / "onnx_models" / "ocr.onnx"

print("📂 Project root:", PROJECT_ROOT)
print("🎥 Video:", VIDEO_PATH)
print("🔠 OCR model:", OCR_ONNX)

assert VIDEO_PATH.exists(), "❌ Video not found"
assert OCR_ONNX.exists(), "❌ OCR ONNX not found"

# =====================================================
# LOAD OCR MODEL
# =====================================================
providers = ["CUDAExecutionProvider", "CPUExecutionProvider"]
ocr_sess = ort.InferenceSession(str(OCR_ONNX), providers=providers)
ocr_in = ocr_sess.get_inputs()[0].name

ALPHABET = "0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ"

# =====================================================
# OCR HELPERS (ROBUST)
# =====================================================
def softmax(x):
    x = x - x.max(axis=-1, keepdims=True)
    e = np.exp(x)
    return e / e.sum(axis=-1, keepdims=True)

def decode_ctc(logits):
    probs = softmax(logits)[0]
    seq = probs.argmax(axis=-1)
    prev = -1
    txt, conf = "", []
    for t, c in enumerate(seq):
        if c != prev and c > 0 and c <= len(ALPHABET):
            txt += ALPHABET[c-1]
            conf.append(probs[t, c])
        prev = c
    return txt, float(np.mean(conf)) if conf else 0.0

def preprocess_big_text(roi):
    """
    Designed specifically for BLUE wagon + WHITE text
    """
    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

    # suppress blue background
    mask = cv2.inRange(hsv, (90, 50, 50), (140, 255, 255))
    roi[mask > 0] = (0, 0, 0)

    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    gray = cv2.normalize(gray, None, 0, 255, cv2.NORM_MINMAX)

    bin_img = cv2.adaptiveThreshold(
        gray, 255,
        cv2.ADAPTIVE_THRESH_MEAN_C,
        cv2.THRESH_BINARY,
        31, 5
    )

    return bin_img

def ocr_from_roi(roi):
    roi = cv2.resize(roi, (128, 32))
    roi = preprocess_big_text(roi)
    inp = (roi / 255.0).astype(np.float32)[None, None]
    logits = ocr_sess.run(None, {ocr_in: inp})[0]
    return decode_ctc(logits)

# =====================================================
# WAGON COUNTING CONFIG (UNCHANGED LOGIC)
# =====================================================
RESIZE_WIDTH = 960
LINE_X_RATIO = 0.5
STRIP_WIDTH = 10
ROI_Y1_RATIO = 0.55
ROI_Y2_RATIO = 0.75
DIFF_THRESHOLD = 18
COOLDOWN_FRAMES = 30
NO_MOTION_FRAMES = 60

# =====================================================
# MAIN
# =====================================================
cap = cv2.VideoCapture(str(VIDEO_PATH))
fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

wagon_index = 0
cooldown = 0
no_motion = 0
prev_strip = None

wagon_frames = defaultdict(list)
results = []

print("🚆 Wagon counting started")

for frame_id in tqdm(range(total_frames), desc="Processing video"):
    ret, frame = cap.read()
    if not ret:
        break

    h, w = frame.shape[:2]
    scale = RESIZE_WIDTH / w
    frame = cv2.resize(frame, (RESIZE_WIDTH, int(h * scale)))
    H, W = frame.shape[:2]

    x = int(W * LINE_X_RATIO)
    y1 = int(H * ROI_Y1_RATIO)
    y2 = int(H * ROI_Y2_RATIO)

    strip = frame[y1:y2, x-STRIP_WIDTH:x+STRIP_WIDTH]
    gray = cv2.cvtColor(strip, cv2.COLOR_BGR2GRAY)

    if prev_strip is None:
        prev_strip = gray
        continue

    diff = cv2.absdiff(gray, prev_strip).mean()
    prev_strip = gray

    if diff > DIFF_THRESHOLD:
        no_motion = 0
        if cooldown == 0:
            wagon_index += 1
            cooldown = COOLDOWN_FRAMES
            print(f"🆕 WAGON #{wagon_index} at frame {frame_id}")
    else:
        no_motion += 1

    if cooldown > 0:
        cooldown -= 1

    if no_motion > NO_MOTION_FRAMES and wagon_index > 0:
        print("🚫 Train exited scene")
        break

    # -------------------------------------------------
    # OCR COLLECTION (MULTI-FRAME PER WAGON)
    # -------------------------------------------------
    if wagon_index > 0:
        roi = frame[int(H*0.45):int(H*0.7), int(W*0.2):int(W*0.8)]
        txt, conf = ocr_from_roi(roi)
        if conf > 0.3 and len(txt) >= 3:
            wagon_frames[wagon_index].append(txt)

    # -------------------------------------------------
    # DISPLAY
    # -------------------------------------------------
    cv2.line(frame, (x, 0), (x, H), (0,0,255), 2)
    cv2.putText(frame, f"Wagons: {wagon_index}", (20,40),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
    cv2.imshow("Wagon Counter + OCR", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# =====================================================
# FINAL AGGREGATION + CSV
# =====================================================
csv_path = PROJECT_ROOT / "wagon_results.csv"
with open(csv_path, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["wagon_index", "ocr_text"])

    for idx in range(1, wagon_index+1):
        texts = wagon_frames[idx]
        if texts:
            text = Counter(texts).most_common(1)[0][0]
        else:
            text = "<NO_TEXT>"
        writer.writerow([idx, text])

print("\n===================================")
print(f"✅ TOTAL WAGONS: {wagon_index}")
print(f"📄 CSV saved to: {csv_path}")
print("===================================")


📂 Project root: c:\Users\Manas Mehta\Desktop\PROJECTS\AIDTM
🎥 Video: c:\Users\Manas Mehta\Desktop\PROJECTS\AIDTM\data\raw_videos\high_speed_axis_55kmph_night.mp4
🔠 OCR model: c:\Users\Manas Mehta\Desktop\PROJECTS\AIDTM\onnx_models\ocr.onnx
🚆 Wagon counting started


Processing video:  36%|███▌      | 1097/3082 [00:13<00:17, 111.00it/s]

🆕 WAGON #1 at frame 1089


Processing video:  37%|███▋      | 1130/3082 [00:14<00:24, 79.36it/s] 

🆕 WAGON #2 at frame 1119


...

🆕 WAGON #57 at frame 2792


Processing video:  92%|█████████▏| 2830/3082 [00:40<00:03, 66.68it/s]

🆕 WAGON #58 at frame 2822


Processing video:  94%|█████████▎| 2888/3082 [00:41<00:02, 69.09it/s]

🚫 Train exited scene

✅ TOTAL WAGONS: 58
📄 CSV saved to: c:\Users\Manas Mehta\Desktop\PROJECTS\AIDTM\wagon_results.csv


In [13]:
%pip install easyocr


   ---------------------------------------- 0.0/2.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.9 MB ? eta -:--:--
   ------- -------------------------------- 0.5/2.9 MB 1.4 MB/s eta 0:00:02
   ---------- ----------------------------- 0.8/2.9 MB 1.8 MB/s eta 0:00:02
   ------------------ --------------------- 1.3/2.9 MB 1.9 MB/s eta 0:00:01
   ------------------------- -------------- 1.8/2.9 MB 2.1 MB/s eta 0:00:01
   -------------------------------- ------- 2.4/2.9 MB 2.3 MB/s eta 0:00:01
   ---------------------------------------- 2.9/2.9 MB 2.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/38.9 MB ? eta -:--:--
    --------------------------------------- 0.8/38.9 MB 3.8 MB/s eta 0:00:11
   - -------------------------------------- 1.6/38.9 MB 3.8 MB/s eta 0:00:10
   -- ------------------------------------- 2.1/38.9 MB 3.3 MB/s eta 0:00:12
   -- ------------------------------------- 2.9/38.9 MB 3.6 MB/s eta 0:00:10
   --- ---------------------

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\Users\\Manas Mehta\\Desktop\\PROJECTS\\gpuenv\\Lib\\site-packages\\cv2\\cv2.pyd'
Check the permissions.


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
import easyocr
reader = easyocr.Reader(['en','hi'])
result = reader.readtext(roi)


ModuleNotFoundError: No module named 'easyocr'